In [11]:
import numpy as np

In [12]:
def tensor_state(str):
    """ Creates a numpy vector representation of a tensor
    product state consisting of |0> 's and |1> 's

    Args:
        string of zeros and ones, e.g. '10'

    Returns:
        np.array([0,1,0,0,...])
    """
    zero, one = np.array([1,0]), np.array([0,1])
    basis_objects = [zero,one]
    objects = [basis_objects[int(str[-1])]]
    for idx in range(0 , len(str) - 1):
        current_object = np.zeros(2**(idx+2))
        counter = 0
        for i in range(2):
            for j in range(len(objects[-1])):
                current_object[counter] = objects[-1][j] * basis_objects[int(str[idx])][i]
                counter += 1
        objects.append(current_object)
    return objects[-1]


In [13]:
class StateVector:
    def __init__(self,number_list):
        self.vector = self._set_vector(number_list)

    @staticmethod
    def _set_vector(digit_list):
        return np.array(digit_list).reshape(len(digit_list),1)

    @staticmethod
    def transpose(self):
        self.vector = np.transpose(self.vector)

    def tensor_with(self,vec2):
        assert type(vec2) is StateVector, "Give vector as instance of StateVector class"
        if self.vector.shape == vec2.vector.shape:
            result = np.zeros((self.vector.shape[0] * vec2.vector.shape[0],1))
            if self.vector.shape[0] > self.vector.shape[1]:
                idx = 0
                for i in range(self.vector.shape[0]):
                    for j in range(vec2.vector.shape[0]):
                        result[idx] = self.vector[i] * vec2.vector[j]
                        idx += 1
            return result
        if self.vector.shape[0] > vec2.vector.shape[0]:
            result = np.zeros((self.vector.shape[0] , vec2.vector.shape[1]))
            for i in range(self.vector.shape[0]):
                for j in range(vec2.vector.shape[1]):
                    result[i][j] = self.vector[i] * vec2.vector[0][j]
            return result
        if vec2.vector.shape[0] > self.vector.shape[0]:
            result = np.zeros((vec2.vector.shape[0] , self.vector.shape[1]))
            for i in range(vec2.vector.shape[0]):
                for j in range(self.vector.shape[1]):
                    result[i][j] = vec2.vector[i] * self.vector[0][j]
            return result

class OperatorMatrix:

    def __init__(self,string):
        self.operator_string = string
        self.operator_matrix = self._multiple_matrix_tensor()

    def _fill_area(self, start_row, start_col, target, recipe, multiplier):
        """ Hepler function for filling out subarea of target matrix
            with content of recipe * multiplier
        Args:
            start_row : Integer = The beginning row index of area
            start_col : Integer = The beginning col index of area
            target    : 2D numpy array to be filled in subarea
            recipe    : 2D numpy array that goes into subarea of target
            multiplier: Float (possibly complex) multiplied onto all recipe vals
        """
        row_counter = 0
        for row in range(start_row,start_row + 2):
            col_counter = 0
            for col in range(start_col, start_col + 2):
                target[row][col] = recipe[row_counter][col_counter] * multiplier
                col_counter += 1
            row_counter+=1

    def _multiple_matrix_tensor(self):

        complex_vals = False
        if "Y" in self.operator_string: complex_vals = True
        if "H" in self.operator_string: print(r"NOTICE: 1/√(2) factor is omitted from Hadamard")

        sigma_x, sigma_y, sigma_z = np.array([[0,1],[1,0]]),np.array([[0,-1j],[1j,0]]),np.array([[1,0],[0,-1]])
        identity = np.identity(2)
        hadamard = np.array([[1,1],[1,-1]])
        basis_map = {"X":sigma_x,"Y":sigma_y,"Z":sigma_z,"I":identity,"H":hadamard}

        objects = [basis_map[self.operator_string[0]]]
        for idx in range(0 , len(self.operator_string) - 1):
            if complex_vals:
                current_object = np.zeros((2 ** (idx+2) , 2 ** (idx+2)),dtype=complex)
            else:
                current_object = np.zeros((2 ** (idx+2) , 2 ** (idx+2)))
            row_counter = 0
            for i in range(int(current_object.shape[0] / 2)):
                col_counter = 0
                for j in range(int(current_object.shape[0] / 2)):
                    if objects[-1][i][j] == 1:
                        self._fill_area(row_counter,col_counter,current_object,basis_map[self.operator_string[idx+1]],1)
                    elif objects[-1][i][j] == -1:
                        self._fill_area(row_counter,col_counter,current_object,basis_map[self.operator_string[idx+1]],-1)
                    elif objects[-1][i][j] == 1j:
                        self._fill_area(row_counter,col_counter,current_object,basis_map[self.operator_string[idx+1]],1j)
                    elif objects[-1][i][j] == -1j:
                        self._fill_area(row_counter,col_counter,current_object,basis_map[self.operator_string[idx+1]],-1j)
                    col_counter += 2
                row_counter += 2
            objects.append(current_object)
        return objects[-1]

In [14]:
initial_state = StateVector([1,2,3])
print(initial_state.vector)

[[1]
 [2]
 [3]]


In [15]:
stateA = StateVector([1,2])
stateB = StateVector([1,2])
print(stateA.tensor_with(stateB))

[[1.]
 [2.]
 [2.]
 [4.]]


In [18]:
IH = OperatorMatrix("IH"); HI = OperatorMatrix("HI"); HH = OperatorMatrix("HH")
print(IH.operator_matrix); print(HI.operator_matrix), print(HH.operator_matrix)

NOTICE: 1/√(2) factor is omitted from Hadamard
NOTICE: 1/√(2) factor is omitted from Hadamard
NOTICE: 1/√(2) factor is omitted from Hadamard
[[ 1.  1.  0.  0.]
 [ 1. -1.  0.  0.]
 [ 0.  0.  1.  1.]
 [ 0.  0.  1. -1.]]
[[ 1.  0.  1.  0.]
 [ 0.  1.  0.  1.]
 [ 1.  0. -1. -0.]
 [ 0.  1. -0. -1.]]
[[ 1.  1.  1.  1.]
 [ 1. -1.  1. -1.]
 [ 1.  1. -1. -1.]
 [ 1. -1. -1.  1.]]


(None, None)

In [17]:
zero_zero = StateVector(tensor_state("00"))
zero_one  = StateVector(tensor_state("01"))
one_zero  = StateVector(tensor_state("10"))
one_one   = StateVector(tensor_state("11"))

In [23]:
print(HH.operator_matrix @ zero_one.vector)

[[ 1.]
 [-1.]
 [ 1.]
 [-1.]]


In [21]:
print(zero_zero.vector-zero_one.vector+one_zero.vector-one_one.vector)

[[ 1.]
 [-1.]
 [ 1.]
 [-1.]]


In [24]:
zero = StateVector(tensor_state("0")); one  = StateVector(tensor_state("1"))
print(zero.vector); print(one.vector)

[[1]
 [0]]
[[0]
 [1]]


In [25]:
print(zero.vector + one.vector)

[[1]
 [1]]


In [26]:
print(zero.vector - one.vector)

[[ 1]
 [-1]]
